In [1]:
#With the network of this notebook we obtained a result of 72,4% on the test set

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_anode_interactivity = "all"

In [2]:
#Imports and we set the seed in order to make the experiment replicable

import os
import tensorflow as tf
import numpy as np

SEED = 1234
tf.random.set_seed(SEED)  
np.random.seed(SEED)

In [ ]:
# Run this cell only if you are using Colab with Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/My Drive/data.zip"

In [3]:
# ImageDataGenerator: we apply some data augmentation in order to somehow face the scarsity of the training set
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

apply_data_augmentation = True

# Create training ImageDataGenerator object
if apply_data_augmentation:
    train_data_gen = ImageDataGenerator(rotation_range=10,
                                        width_shift_range=10,
                                        height_shift_range=10,
                                        zoom_range=0.3,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)

# Create validation and test ImageDataGenerator objects
valid_data_gen = ImageDataGenerator(rescale=1./255)


In [4]:
# Create generators to read images from dataset directory
# -------------------------------------------------------
# Get current working directory
cwd = os.getcwd()
dataset_dir = os.path.join(cwd, 'data')

# img

# Batch size
bs = 32

# img shape
img_h = 256
img_w = 256

num_classes=3

# Training
training_dir = os.path.join(dataset_dir, 'training')
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               shuffle=True,
                                               target_size=(img_h, img_w),
                                               seed=SEED)  # targets are directly converted into one-hot vectors

# Validation
validation_dir = os.path.join(dataset_dir, 'validation')
valid_gen = valid_data_gen.flow_from_directory(validation_dir,
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               shuffle=False,
                                               target_size=(img_h, img_w),
                                               seed=SEED)


Found 5052 images belonging to 3 classes.
Found 562 images belonging to 3 classes.


In [5]:
# Create Dataset objects
# ----------------------
size = (256, 256)
# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Shuffle (Already done in generator..)
# train_dataset = train_dataset.shuffle(buffer_size=len(train_gen))

# Normalize images (Already done in generator..)
# def normalize_img(x_, y_):
#     return tf.cast(x_, tf.float32) / 255., y_

# train_dataset = train_dataset.map(normalize_img)

# 1-hot encoding <- for categorical cross entropy (Already done in generator..)
# def to_categorical(x_, y_):
#     return x_, tf.one_hot(y_, depth=10)

# train_dataset = train_dataset.map(to_categorical)

# Divide in batches (Already done in generator..)
# train_dataset = train_dataset.batch(bs)

# Repeat
# Without calling the repeat function the dataset 
# will be empty after consuming all the images
#Resize performed here both for training and validation
train_dataset = train_dataset.map(lambda x, y: (tf.image.resize(x, size), y))
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
valid_dataset = valid_dataset.map(lambda x, y: (tf.image.resize(x, size), y))
valid_dataset = valid_dataset.repeat()



In [6]:
# Architecture: Features extraction -> Classifier
#Features extraction: we have 6 blocks with an initial number of filters that is 16, it's doubled in each block.
#Each block has a conv2D, an activation and a MaxPool2D, we tried to improve the complexity of this part adding more blocks
#and filters in order to obtain better features to feed the last part.
#Classifier: we subistituted the fully connected part with a GAP layer linked to the dense with the softmax and
#we still got better results.

start_f = 16
depth = 6

model = tf.keras.Sequential()

# Features extraction
for i in range(depth):

    if i == 0:
        input_shape = [img_h, img_w, 3]
    else:
        input_shape=[None]

    # Conv block: Conv2D -> Activation -> Pooling
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                     kernel_size=(3, 3),
                                     strides=(1, 1),
                                     padding='same',
                                     input_shape=input_shape))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))
    start_f *= 2
    
# Classifier
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 16)      448       
_________________________________________________________________
re_lu (ReLU)                 (None, 256, 256, 16)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 128, 128, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        1

In [7]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

import os
from datetime import datetime



cwd = os.path.join(os.getcwd(),"drive/My Drive/logs")

exps_dir = os.path.join(cwd, 'custom_1')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'custom'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

#ReduceLROnPlateau
LROnPlateau = False
if LROnPlateau:
    LROP_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", verbose=1,mode="auto")
    callbacks.append(LROP_callback)
    


# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True )
    callbacks.append(es_callback)

epochs = 80
model.fit(x=train_dataset, epochs=epochs,steps_per_epoch=len(train_gen),
          validation_data=valid_dataset,
          validation_steps=len(valid_gen), 
          callbacks=callbacks)

Train for 158 steps, validate for 18 steps
Epoch 1/80
158/158 [==============================] - 136s 861ms/step - loss: 1.0995 - accuracy: 0.3325 - val_loss: 1.0982 - val_accuracy: 0.3470
Epoch 2/80
158/158 [==============================] - 100s 633ms/step - loss: 1.0981 - accuracy: 0.3502 - val_loss: 1.0995 - val_accuracy: 0.3416
Epoch 3/80
158/158 [==============================] - 109s 688ms/step - loss: 1.0982 - accuracy: 0.3304 - val_loss: 1.0971 - val_accuracy: 0.3754
Epoch 4/80
158/158 [==============================] - 96s 606ms/step - loss: 1.0977 - accuracy: 0.3484 - val_loss: 1.0982 - val_accuracy: 0.3452
Epoch 5/80
158/158 [==============================] - 113s 717ms/step - loss: 1.0979 - accuracy: 0.3494 - val_loss: 1.0956 - val_accuracy: 0.3772
Epoch 6/80
158/158 [==============================] - 97s 613ms/step - loss: 1.0959 - accuracy: 0.3709 - val_loss: 1.1010 - val_accuracy: 0.3470
Epoch 7/80
158/158 [==============================] - 110s 698ms/step - loss: 1.093

In [8]:
import pandas as pd
from datetime import datetime

def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')


test_dir = os.path.join(dataset_dir, 'test')

images = [f for f in os.listdir(test_dir)]
images = pd.DataFrame(images)
images.rename(columns = {0:'filename'}, inplace = True)
images["class"] = 'test'

test_gen = train_data_gen.flow_from_dataframe(images,
                                               test_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)


test_gen.reset()

predictions = model.predict_generator(test_gen, len(test_gen), verbose=1)

results = {}
images = test_gen.filenames
i = 0

for p in predictions:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i = i + 1

create_csv(results)

Found 450 validated image filenames belonging to 1 classes.
Instructions for updating:
Please use Model.predict, which supports generators.
15/15 [==============================] - 9s 583ms/step
